In [229]:
import openpyxl
import pprint
import re
import pandas as pd

#Excel読み込み
wb1 = openpyxl.load_workbook("moto_sample.xlsx")
wb2 = openpyxl.load_workbook("seikyu.xlsx")

#シートネーム全部リスト化
wb1sheet = [sheetname for sheetname in wb1.sheetnames]#請求元サンプル入ってる。
wb2sheet = [sheetname for sheetname in wb2.sheetnames]#請求書一覧データ

sheet1 = wb2["請求一覧2101"]
#3行目をカラムとしてリスト化
col = []
for row in sheet1.iter_cols():
    for cell in row:
        if cell.row == 3:
            col.append(cell.value)
print(col)

#各シートデータ、行最大値リスト化
sheet_list = []
max_row_list = []
for i,sheet in enumerate(wb2sheet):
    if re.search('請求一覧', sheet):
        sheet_row = wb2[wb2sheet[i]]
        sheet_list.append(sheet_row)
        max_row_list.append(sheet_row.max_row)

#列名置き換え用辞書型カラムデータ作成
dic_col = {}
for i,col in enumerate(col):
    dic_col[i] = col
 
#print(dic_col)
#print(sheet_list)
#print(max_row_list)

#行情報取得関数（取得した行をリストに入れる）
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                          max_col=end_col))

#顧客管理テーブルシートを読み込んでデータフレーム化
cs = wb2["顧客管理テーブル"]
cs_list = get_list_2d(cs,3,cs.max_row,1,2)

cs_df = pd.DataFrame(cs_list)
cs_df = cs_df.dropna(how='all')
#請求一覧と合体させたいので顧客ＣＤを相手先コードにする
cs_df.columns = ['相手先コード','顧客名']
#display(cs_df)


['受注No.', '相手先名', '相手先コード', '商品名１', '商品名２', '検収日', '金額（税抜き）', '備考']


In [230]:
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import calendar

seikyu_all = pd.DataFrame()
for i,sheet in enumerate(sheet_list):
    l_2d = get_list_2d(sheet, 4, max_row_list[i], 0, 8)
    name = sheet.title[:4]
    year = "20" + sheet.title[4:6]
    month = sheet.title[6:]
    _, lastday = calendar.monthrange(int(year),int(month))
    date = datetime(int(year),int(month),lastday)
    date_ym = date.strftime('%Y-%m-%d')
    df = pd.DataFrame(l_2d)
    seikyu_df = df.rename(columns=dic_col)
    seikyu_df = seikyu_df.dropna(how='all')
    seikyu_df['年月'] = date_ym
    #seikyu_df = seikyu_df.set_index('年月',drop = True)
    seikyu_all = pd.concat([seikyu_all, seikyu_df])

#display(seikyu_all)

seikyu_sitei = pd.DataFrame()

#seikyu_sitei = seikyu_all[(seikyu_all["相手先コード"] == 1005) &  (seikyu_all["年月"] == "2021-02")]
#seikyu_sitei = seikyu_all[seikyu_all["相手先コード"] == 1005]

#請求一覧、顧客一覧データフレームをマージする。
merge_df = pd.merge(seikyu_all,cs_df)
nen_df = merge_df["年月"]
nen_df = nen_df[~nen_df.duplicated()]
nen_df = nen_df.sort_values()
cs_code_df = cs_df["相手先コード"]
display(nen_df)
display(cs_code_df)

#相手先コード、年月からデータを抽出し、受注No.でソートしたものをデータフレームへ代入   
merge_df = merge_df[(merge_df["相手先コード"] == 1003) & (merge_df["年月"] == "2021-07-31")].sort_values(by="受注No.",ascending = True)
#年月
date = merge_df["年月"].iloc[0]
date_excel = date.replace("-","/")
#No
no = date[2:4] + date[5:7] + "01"
#会社名
name = merge_df["顧客名"].iloc[0] + "　御中"
file_name = merge_df["顧客名"].iloc[0]
print(no,date,name)

#各行最大値
maxRow = moto_sheet.max_row + 1
#最大列
maxClm = moto_sheet.max_column + 1
#列ループ
for j in range(1,maxClm):

    #行を逆ループ
    for i in reversed(range(1,maxRow)):

        #セルがNoneではなければ
        if moto_sheet.cell(row=i, column=j).value != None:

            #表示
            print(str(j) + '列目の最終行：' + str(i))
            if j == 5:
                goukei = i
            break


#請求元シートへデータを書き込んで、出力する。

#請求元シート読み込み
moto_sheet = wb1["請求書元"]
moto_sheet['H1'] = date_excel
moto_sheet['H2'] = no
moto_sheet['A6'] = name
col_list = ['受注No.','商品名１','商品名２','数量','単位','金額（税抜き）','備考']

#請求元シートへ各データ書き込み
TAX = 1.08

# B2~F2のセルを結合
#moto_sheet[].merge_cells(range_string="B2:F2")
# B2セルに［中央揃え］を設定
#moto_sheet["B2"].alignment = Alignment(horizontal="center")

                #if cell.value is None:
                    #mono1 = "B" + str(excel_num - 1) + ":" + "D" + str(excel_num - 1)
                    #mono2 = "B" + str(excel_num) + ":" + "D" + str(excel_num)
                    #mono3 = "B" + str(excel_num - 1) + ":" + "D" + str(excel_num)
                    #moto_sheet.unmerge_cells(mono1)
                    #moto_sheet.unmerge_cells(mono2)
                    #moto_sheet.merge_cells(mono3)   

for i,col in enumerate(col_list):
    try:
        if col == "受注No.":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "A" + str(excel_num)
                print(cell_num,data)
                moto_sheet[cell_num] = data
        elif col == "商品名１":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "B" + str(excel_num)
                print(cell_num,data)
                moto_sheet[cell_num] = data
                #moto_sheet[cell_num].alignment = Alignment(horizontal="left")
        elif col == "商品名２":
            for i,data in enumerate(merge_df[col]):
                excel_num = 19 + (i*2)
                cell_num = "B" + str(excel_num)
                print(cell_num,data)
                moto_sheet[cell_num] = data
                cell = moto_sheet[cell_num]        
        elif col == "金額（税抜き）":
            sum = 0
            excel_num = 18
            for i,data in enumerate(merge_df[col]):
                excel_num = excel_num + (i*2)
                cell_num = "G" + str(excel_num)
                print(cell_num,data)
                moto_sheet[cell_num] = data
                sum += data
            
            excel_num += 2
            cell_num = "G" + str(excel_num)
            zei = (sum * TAX) - sum
            moto_sheet[cell_num] = zei
            cell_num = "B" + str(excel_num + 1)
            moto_sheet[cell_num] = "以上に掛かる消費税"
            #moto_sheet[cell_num].alignment = Alignment(horizontal="right")
            cell_num = "B" + str(excel_num + 2)
            moto_sheet[cell_num] = "～　以　下　余　白　～"
            #moto_sheet[cell_num].alignment = Alignment(horizontal="center")
            cell_num = "G" + str(goukei)
            moto_sheet[cell_num] = sum * TAX

            
        elif col == "備考":
            for i,data in enumerate(merge_df[col]):
                excel_num = 18 + (i*2)
                cell_num = "H" + str(excel_num)
                print(cell_num,data)
                moto_sheet[cell_num] = data
    except:
        continue

print(date,type(date))
#before = datetime.strptime(date, '%Y-%m-%d')
#after = before + timedelta(month = 1)


print(after)
#各行最大値
maxRow = moto_sheet.max_row + 1
#最大列
maxClm = moto_sheet.max_column + 1
#列ループ
for j in range(1,maxClm):

    #行を逆ループ
    for i in reversed(range(1,maxRow)):

        #セルがNoneではなければ
        if moto_sheet.cell(row=i, column=j).value != None:

            #表示
            print(str(j) + '列目の最終行：' + str(i))
            if j == 5:
                goukei = i
            break

#末尾サンプルシート削除
wb1.remove(wb1.worksheets[-1])
#シートネーム変更
moto_sheet.title = date + "請求書"
#シート新規保存
wb1.save(merge_df["顧客名"].iloc[0] + date + '.xlsx')

0     2021-01-31
1     2021-02-28
2     2021-03-31
7     2021-05-31
11    2021-06-30
16    2021-07-31
4     2021-08-31
18    2021-09-30
6     2021-10-31
Name: 年月, dtype: object

0     1001.0
1     1002.0
2     1003.0
3     1004.0
4     1005.0
5     2001.0
6     2002.0
7     3001.0
8     3002.0
9     4001.0
10    5001.0
11    5002.0
12    6002.0
Name: 相手先コード, dtype: float64

210701 2021-07-31 プロト技研株式会社　御中
1列目の最終行：35
2列目の最終行：33
3列目の最終行：32
4列目の最終行：3
5列目の最終行：28
6列目の最終行：17
7列目の最終行：28
8列目の最終行：20
A18 210402.0
A20 210403.0
B18 バッテリー制御システム
B20 バッテリー制御システム
B19 bmc064、068
B21 bmc069、071
G18 387000.0
G20 354000.0
H18 発注書毎請求
H20 発注書毎請求
2021-07-31 <class 'str'>


TypeError: 'month' is an invalid keyword argument for this function